In [36]:
import os
import itertools
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import folds
import utils

In [5]:
'''Code to view category prevalence'''
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
df = pd.read_csv(os.path.join(dataset_path, 'HAM10000_metadata'))
df.dx.value_counts()/df.shape[0]

nv       0.669496
mel      0.111133
bkl      0.109735
bcc      0.051323
akiec    0.032651
vasc     0.014179
df       0.011483
Name: dx, dtype: float64

In [10]:
'''Code to create labels.csv'''
def label_to_category(label, categories):
    if label not in categories: return [0]
    else: return [categories.index(label)+1]

dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
df = pd.read_csv(os.path.join(dataset_path, 'HAM10000_metadata'))
categories = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
categories = ['bkl', 'nv', 'mel'] # categories based off prevalence
df['label'] = df['dx'].apply(lambda label: label_to_category(label, categories))
df['path'] = df['image_id'].apply(lambda image_id: os.path.join(dataset_path, 'train/{}.jpg'.format(image_id)))
temp_df = df[['lesion_id', 'label', 'path']].set_index('lesion_id')
temp_df.to_csv(os.path.join(dataset_path, 'train/labels.csv'))
temp_df.head()

,label,path
lesion_id,,
HAM_0000118,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0000118,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0002730,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0002730,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...
HAM_0001466,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tra...


In [30]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
# TODO: Remove ISIC_0035068.jpg from ISIC2018_Task3_Test_GroundTruth.csv,
#       the image is missing from ISIC2018_Task3_Test_Images.zip
df = pd.read_csv(os.path.join(dataset_path, 'ISIC2018_Task3_Test_GroundTruth.csv'))
categories = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']
categories = ['bkl', 'nv', 'mel'] # categories based off prevalence
df['label'] = df['dx'].apply(lambda label: label_to_category(label, categories))
df['path'] = df['image_id'].apply(lambda image_id: os.path.join(dataset_path, 'test/{}.jpg'.format(image_id)))
temp_df = df[['lesion_id', 'label', 'path']].set_index('lesion_id')
temp_df.to_csv(os.path.join(dataset_path, 'test/labels.csv'))
temp_df.head()

,label,path
lesion_id,,
HAMTEST_0000000,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000001,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000002,[1],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000003,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...
HAMTEST_0000004,[2],/cluster/tufts/hugheslab/eharve06/HAM10000/tes...


In [65]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
n = 100
tune = False
random_state = 3001
augmented_train_dataset, train_dataset, val_or_test_dataset = utils.get_ham10000_datasets(dataset_path, n, tune=tune, random_state=random_state)
print(len(augmented_train_dataset))
print(len(train_dataset))
print(len(val_or_test_dataset))

100
100
1511


In [66]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/HAM10000'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000'
lr_0s = np.logspace(-1, -4, num=4)
ns = [100, 1000, 10000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
random_states = [1001, 2001, 3001]
prior_type = 'adapted'
wandb_project = 'tuned_HAM10000'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
    print('    "python ../src/HAM10000_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.01' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.01"
    "python ../src/HAM10000_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/HAM10000' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_HAM10000' --lr_0=0.1 --model_name='adapted_lr_0=0.1_n=100_random_state=1001_weight_decay=0.001' --n=100 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_HAM10000' --weight_decay=0.001"
    "python ../src/HAM10000_main.py --dataset_path='